### ****** 필독 !! 실습과제 주의사항 ******

- 각자의 "수강생 번호" 확인 (아이캠퍼스 공지) 
- 제출하는 실습과제 파일에 "수강생 번호"를 기준으로 작성 (이름, 학번 등 작성X)
- 각 실습과제에 대한 구체적인 파일 이름은 매번 개별 안내 
  (수강생 번호 123번 학생 과제파일 예시 : 'ST123_HW3_1' )
  
- 과제 파일이름 양식 지키지 않을 시 감점!

# [실습 과제 3]

## ■ LearningX 과목의 10/24일 공지 필독 후 과제 진행 (내용 반영되지 않은 상태로 수행한 과제는 감점!)

## KNN, SVM, ANN 모델 각각 <font color=red>Fold 3 학습/검증 데이터</font>만 사용할 때 기준으로 Hyperparameter 별 모델 성능 확인하기!
### * <font color=red>sklearn.model_selection.GridSearchCV 함수 사용 X</font>, 반복문 사용
### * 검증 정확도 기준 정렬하여 Hyperparameter 조합별 <font color=red>검증정확도 결과</font> 제출
### * <font color=red>검증정확도 결과</font>의 column 이름 및 순서는 실습코드와 동일하게 구성

#### >>>>>> KNN Hyperparameter 아래 참조 (다른 파라미터는 전부 기본값으로)
#### >>>>>>>>>> weight : uniform, distance
#### >>>>>>>>>> n_neighbors : 3, 5, 7
#### >>>>>>>>>> metric : manhattan, euclidean

#### >>>>>> SVM Hyperparameter 아래 참조 (다른 파라미터는 전부 기본값으로)
#### >>>>>>>>>> kernel : poly, rbf
#### >>>>>>>>>> C : 0.001, 0.01, 0.1, 1
#### >>>>>>>>>> gamma : 0.01, 0.1, 1

#### >>>>>> ANN Hyperparameter 아래 참조 (iteration 500 고정, 다른 파라미터는 전부 기본값으로, Keras 사용하기)
#### >>>>>>>>>> Learning rate : 0.0001, 0.001, 0.01
#### >>>>>>>>>> Number of hidden layer : 1, 2
#### >>>>>>>>>> Number of neurons per layer : 2, 4, 8

###  각 모델의 조합별 검증정확도를 (확장자명 없는) csv파일로 제출 
#### >> KNN 정확도 결과물 : ST(수강생 번호)_HW3_1
#### >> SVM 정확도 결과물 : ST(수강생 번호)_HW3_2
#### >> ANN 정확도 결과물 : ST(수강생 번호)_HW3_3  
#### 주의 : 변수명.to_csv 함수에서 <font color=red>header=None, index=None 적용 금지. (제출한 파일을 불러왔을 시 상단 column명에 Hyperparameter 종류가 표시되어야 함) </font>

## ***** 검증정확도 결과물 zip파일로 압축하여 제출
### >>> 압축파일 이름 ST(수강생 번호)_HW3 (예시 : 'ST00_HW3'  //  'ST0_HW3')

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import pickle
from sklearn.neighbors import KNeighborsClassifier
import joblib
from sklearn.model_selection import GridSearchCV

In [2]:
Fold = 5

# k-fold 학습/검증 데이터
for i in range(Fold):
    
    path1 = './K_FoldData/Training_Fold%d'%(i+1)
    path2 = './K_FoldData/Validation_Fold%d'%(i+1)
    c1 = 'Training_Fold%d   = np.array(pd.read_csv(path1, sep=",", header=None))'%(i+1)
    c2 = 'Validation_Fold%d = np.array(pd.read_csv(path2, sep=",", header=None))'%(i+1)
    exec(c1)
    exec(c2)

# K-fold 학습/검증 레이블
TrainingFold_Label   = np.array(pd.read_csv('./K_FoldData/TrainingFold_Label'  , sep=",", header=None).T.squeeze())
ValidationFold_Label = np.array(pd.read_csv('./K_FoldData/ValidationFold_Label', sep=",", header=None).T.squeeze())
    
    
# 전체 학습용 데이터
Training_All       = np.array(pd.read_csv('./K_FoldData/Training_All', sep = ",", header = None))
Training_All_Label = np.array(pd.read_csv('./K_FoldData/Training_All_Label', sep = ",", header = None).T.squeeze())


# 평가 데이터
Test_Data  = np.array(pd.read_csv('./K_FoldData/Test_Data' , sep=",", header=None))
Test_Label = np.array(pd.read_csv('./K_FoldData/Test_Label', sep=",", header=None))

In [3]:
param_weight      = ['uniform', 'distance']                 # 데이터 간 거리에 따른 가중치 유무
param_n_neighbors = [3, 5, 7]                  # 계산할 주변(이웃) 데이터 개수
param_metric      = ['manhattan','euclidean'] # 거리 계산 방법

# 파라미터별 정확도 기록할 변수 설정
Accuracy_df = pd.DataFrame(np.zeros(shape=(len(param_n_neighbors)*len(param_weight)*len(param_metric),4)), #2*5*3
                           columns=['weight', 'n_neighbors', 'metric', 'Accuracy'])
cnt = 0 #폴드 들어갈때 카운트
Fold= 1

# Grid Search 코드
for Weight in param_weight: #리스트에 있는 원소 하나하나다 넣어주면서 감

    for Nnb in param_n_neighbors: #변수들이 할당해서 for문 안에서 돌아간다

        for Metric in param_metric:
            
            ValidScore = 0 # 모델 바뀔 때마다 검증정확도 변수 초기화
            Training_CurrentFold = Training_Fold3
            Validation_CurrentFold = Validation_Fold3
            tempsvmModel   = KNeighborsClassifier(n_neighbors=Nnb,
                                                      weights=Weight,
                                                      metric=Metric).fit(Training_CurrentFold, TrainingFold_Label)
            tempValidScore = tempsvmModel.score(Validation_CurrentFold, ValidationFold_Label)
            ValidScore += tempValidScore
            Acc = tempValidScore # 평균 검증 정확도
            Accuracy_df.iloc[cnt, :] = [Weight, Nnb, Metric, Acc] # 모델의 하이퍼파라미터, 검증정확도 대입
            cnt += 1

In [4]:
Accuracy_df

,weight,n_neighbors,metric,Accuracy
0,uniform,3.0,manhattan,1.0
1,uniform,3.0,euclidean,1.0
2,uniform,5.0,manhattan,1.0
3,uniform,5.0,euclidean,1.0
4,uniform,7.0,manhattan,1.0
5,uniform,7.0,euclidean,1.0
6,distance,3.0,manhattan,1.0
7,distance,3.0,euclidean,1.0
8,distance,5.0,manhattan,1.0
9,distance,5.0,euclidean,1.0


In [5]:
Accuracy_df_sorted = Accuracy_df.sort_values(by = ['Accuracy'], ascending = False) # 성능 확인 내림차순 정리
Accuracy_df_sorted

,weight,n_neighbors,metric,Accuracy
0,uniform,3.0,manhattan,1.0
1,uniform,3.0,euclidean,1.0
2,uniform,5.0,manhattan,1.0
3,uniform,5.0,euclidean,1.0
4,uniform,7.0,manhattan,1.0
5,uniform,7.0,euclidean,1.0
6,distance,3.0,manhattan,1.0
7,distance,3.0,euclidean,1.0
8,distance,5.0,manhattan,1.0
9,distance,5.0,euclidean,1.0


In [6]:
path = './Subfolder//ST41_HW3_1'   # path = '파일 경로//저장할 파일 이름'
Accuracy_df_sorted.to_csv(path, sep=',')

In [7]:
from sklearn import svm

In [8]:
Fold = 5

# k-fold 학습/검증 데이터
for i in range(Fold):
    
    path1 = './K_FoldData/Training_Fold%d'%(i+1)
    path2 = './K_FoldData/Validation_Fold%d'%(i+1)
    c1 = 'Training_Fold%d   = np.array(pd.read_csv(path1, sep=",", header=None))'%(i+1)
    c2 = 'Validation_Fold%d = np.array(pd.read_csv(path2, sep=",", header=None))'%(i+1)
    exec(c1)
    exec(c2)

# K-fold 학습/검증 레이블
TrainingFold_Label   = np.array(pd.read_csv('./K_FoldData/TrainingFold_Label'  , sep=",", header=None).T.squeeze())
ValidationFold_Label = np.array(pd.read_csv('./K_FoldData/ValidationFold_Label', sep=",", header=None).T.squeeze())
    
    
# 전체 학습용 데이터
Training_All       = np.array(pd.read_csv('./K_FoldData/Training_All', sep = ",", header = None))
Training_All_Label = np.array(pd.read_csv('./K_FoldData/Training_All_Label', sep = ",", header = None).T.squeeze())


# 평가 데이터
Test_Data  = np.array(pd.read_csv('./K_FoldData/Test_Data' , sep=",", header=None))
Test_Label = np.array(pd.read_csv('./K_FoldData/Test_Label', sep=",", header=None))

In [9]:
Fold = 5

# 비교할 하이퍼파라미터들 리스트 형태로 만들기
param_kernel = ['poly', 'rbf'] # hyperplane 모양 rbf 고차원 분포에 유효
param_C      = [0.001, 0.01, 0.1, 1]              # 이상치 허용 정도 c 값이 작아질 수로 이상치를 크게 허용 정제된 데이터의 경우 c파라미터 높게
param_gamma  = [0.01, 0.1, 1]              # hyperplane 곡률(얼마나 가프르게 만들어지냐 높으면 급한느낌) 선형함수에 적용x

# 파라미터별 정확도 기록할 변수 설정
Accuracy_df = pd.DataFrame(np.zeros(shape=(len(param_kernel)*len(param_C)*len(param_gamma),4)),
                           columns=['kernel', 'C', 'gamma', 'Accuracy'])
cnt = 0

# Grid Search 코드
for Kernel in param_kernel:
    
    for c in param_C:
        
        for Gamma in param_gamma:
            
            ValidScore = 0 # 모델 바뀔 때마다 검증정확도 변수 초기화
            Training_CurrentFold = Training_Fold3
            Validation_CurrentFold = Validation_Fold3
            tempsvmModel   = svm.SVC(kernel=Kernel,
                                         C=c,
                                         gamma=Gamma).fit(Training_CurrentFold, TrainingFold_Label) # 학습
            tempValidScore = tempsvmModel.score(Validation_CurrentFold, ValidationFold_Label)   # 검증
            ValidScore += tempValidScore
            Acc = ValidScore # 평균 검증 정확도
            Accuracy_df.iloc[cnt, :] = [Kernel, c, Gamma, Acc] # 모델의 하이퍼파라미터
            cnt += 1

In [10]:
Accuracy_df

,kernel,C,gamma,Accuracy
0,poly,0.001,0.01,0.968750
1,poly,0.001,0.10,1.000000
2,poly,0.001,1.00,1.000000
3,poly,0.010,0.01,0.968750
4,poly,0.010,0.10,1.000000
5,poly,0.010,1.00,1.000000
6,poly,0.100,0.01,0.984375
7,poly,0.100,0.10,1.000000
8,poly,0.100,1.00,1.000000
9,poly,1.000,0.01,1.000000


In [11]:
Accuracy_df_sorted = Accuracy_df.sort_values(by = ['Accuracy'], ascending = False) # 성능 확인 내림차순 정리
Accuracy_df_sorted

,kernel,C,gamma,Accuracy
23,rbf,1.000,1.00,1.000000
19,rbf,0.100,0.10,1.000000
17,rbf,0.010,1.00,1.000000
16,rbf,0.010,0.10,1.000000
14,rbf,0.001,1.00,1.000000
13,rbf,0.001,0.10,1.000000
1,poly,0.001,0.10,1.000000
11,poly,1.000,1.00,1.000000
10,poly,1.000,0.10,1.000000
9,poly,1.000,0.01,1.000000


In [12]:
path = './Subfolder//ST41_HW3_2'   # path = '파일 경로//저장할 파일 이름'
Accuracy_df_sorted.to_csv(path, sep=',')

In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(777) #난수 시드 저장하는거 웨이트랑 바이어스 일정하게 해주기 위함

In [14]:
Fold = 5

# k-fold 학습/검증 데이터
for i in range(Fold):
    
    path1 = './K_FoldData/Training_Fold%d'%(i+1)
    path2 = './K_FoldData/Validation_Fold%d'%(i+1)
    c1 = 'Training_Fold%d   = np.array(pd.read_csv(path1, sep=",", header=None))'%(i+1)
    c2 = 'Validation_Fold%d = np.array(pd.read_csv(path2, sep=",", header=None))'%(i+1)
    exec(c1)
    exec(c2)

# K-fold 학습/검증 레이블
TrainingFold_Label   = np.array(pd.read_csv('./K_FoldData/TrainingFold_Label_forANN'  , sep=",", header=None))
ValidationFold_Label = np.array(pd.read_csv('./K_FoldData/ValidationFold_Label_forANN', sep=",", header=None))

# 전체 학습용 데이터
Training_All       = np.array(pd.read_csv('./K_FoldData/Training_All', sep = ",", header = None))
Training_All_Label = np.array(pd.read_csv('./K_FoldData/Training_All_Label_forANN', sep = ",", header = None))

print(Training_Fold1.shape)
print(Validation_Fold1.shape)
print(TrainingFold_Label.shape)
print(ValidationFold_Label.shape)
print(Training_All.shape)

(256, 30)
(64, 30)
(256, 2)
(64, 2)
(320, 30)


In [15]:
def ANN_model(input_data, learningRate, noOfHiddenLayer, noOfNeuron):
    model = keras.Sequential() #최적화할 파라미터 넣어줌
    model.add(keras.layers.Dense(units = noOfNeuron, input_shape = (input_data.shape[1],) ))   # Input  Layer
    for i in range(noOfHiddenLayer):
        model.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu)) # Hidden Layer 
    model.add(keras.layers.Dense(units = 2,          activation = keras.activations.softmax))  # Output Layer 정상 고장이라 unit=2
    
    model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model

In [16]:
Fold = 5
iteration = 500
Label        = TrainingFold_Label
Label_Val    = ValidationFold_Label

# 비교할 하이퍼파라미터들 리스트 형태로 만들기 파라미터 하나당 2개
param_learningRate      = [0.0001, 0.001,0.01]    # 학습률
param_noOfHiddenLayer   = [1, 2]          # 은닉 레이어 수
param_noOfNeuron        = [2, 4, 8]         # 레이어 당 뉴런 개수

# 파라미터별 정확도 기록할 변수 설정
Accuracy_df = pd.DataFrame(np.zeros(shape=(len(param_learningRate)*len(param_noOfHiddenLayer)*len(param_noOfNeuron),4)),
                           columns=['LearningRate', 'NoOfHiddenLayer', 'NoOfNeuron', 'Accuracy'])
cnt = 0

# Grid Search 코드
for learningRate in param_learningRate:
    
    for noOfHiddenLayer in param_noOfHiddenLayer:
        
        for noOfNeuron in param_noOfNeuron:
            
            ValidScore = 0 # 모델 바뀔 때마다 검증정확도 변수 초기화
            Data = Training_Fold3
            Data_Val = Validation_Fold3
            model = ANN_model(Data, learningRate, noOfHiddenLayer, noOfNeuron)
            tempHist = model.fit(Data, Label, epochs=iteration, verbose = 0)
            tempLoss, tempValidScore = model.evaluate(Data_Val,  Label_Val, verbose=0)
            ValidScore += tempValidScore
            Acc = ValidScore
            Accuracy_df.iloc[cnt, :] = [learningRate, noOfHiddenLayer, noOfNeuron, Acc] # 모델의 하이퍼파라미터
            cnt += 1
            print('{} {} {} model done.'.format(learningRate, noOfHiddenLayer, noOfNeuron))

0.0001 1 2 model done.
0.0001 1 4 model done.
0.0001 1 8 model done.
0.0001 2 2 model done.
0.0001 2 4 model done.
0.0001 2 8 model done.
0.001 1 2 model done.
0.001 1 4 model done.
0.001 1 8 model done.
0.001 2 2 model done.
0.001 2 4 model done.
0.001 2 8 model done.
0.01 1 2 model done.
0.01 1 4 model done.
0.01 1 8 model done.
0.01 2 2 model done.
0.01 2 4 model done.
0.01 2 8 model done.


In [17]:
Accuracy_df_sorted = Accuracy_df.sort_values(by = ['Accuracy'], ascending = False) # 성능 확인 내림차순 정리
Accuracy_df_sorted

,LearningRate,NoOfHiddenLayer,NoOfNeuron,Accuracy
9,0.0010,2.0,2.0,1.00000
6,0.0010,1.0,2.0,1.00000
13,0.0100,1.0,4.0,1.00000
11,0.0010,2.0,8.0,1.00000
8,0.0010,1.0,8.0,1.00000
17,0.0100,2.0,8.0,1.00000
5,0.0001,2.0,8.0,1.00000
2,0.0001,1.0,8.0,1.00000
1,0.0001,1.0,4.0,0.93750
0,0.0001,1.0,2.0,0.90625


In [18]:
path = './Subfolder//ST41_HW3_3'   # path = '파일 경로//저장할 파일 이름'
Accuracy_df_sorted.to_csv(path, sep=',')